In [174]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm as tqdm
import numpy as np

In [175]:
data = pd.read_csv("data.csv")
data = data.dropna()
for i in range(0,data.shape[0]):
    data['fpl_sel'].iloc[i] = float(str(data['fpl_sel'].iloc[i]).rstrip('%'))
data['fpl_sel'] = data['fpl_sel'].astype('float64')
#print(data.dtypes)
#data.head

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [176]:
print(data['page_views'].quantile(0.90))
data["page_views"] = np.where(data["page_views"] <1713.4, 1713.4,data['page_views'])
print(data['fpl_sel'].quantile(0.90))
data["fpl_sel"] = np.where(data["fpl_sel"] <10.4, 10.4,data['fpl_sel'])

1713.400000000001
10.4


In [177]:
import numpy as np
X = data[data.columns[~data.columns.isin(['market_value'])]]
y = data['market_value']

from sklearn.model_selection import train_test_split
np.random.seed(37) # Set seed
x_train, x_test = train_test_split(X, test_size = 0.15, random_state = 40 )

## dont using iloc as we have projected from data
y_train = y.loc[x_train.index.values] 
y_test = y.loc[x_test.index.values]
x_train = X.loc[x_train.index.values, :]
x_test = X.loc[x_test.index.values, :]

In [178]:
concat_train_data = pd.concat([x_train,y_train],axis=1)
x_train = x_train.dropna()
x_train = x_train.drop(['position','name','age','club','nationality'], axis = 1)
#scaling
from sklearn.preprocessing import StandardScaler
scaler =StandardScaler()
scaled_data = scaler.fit_transform(x_train)
#print(scaler.transform(concat_train_data))
scaled_data.shape

(391, 11)

In [179]:

from sklearn.decomposition import PCA
pca = PCA(n_components=11)
pca.fit(scaled_data)
print(pca.explained_variance_)
print(np.cumsum(pca.explained_variance_ratio_*100))

[2.75072243 1.39494706 1.32698917 1.08215631 0.98875446 0.9204883
 0.76665807 0.69144442 0.54600807 0.4097055  0.15033134]
[ 24.94261216  37.59151596  49.62420086  59.43682491  68.40251288
  76.74918657  83.7009803   89.97076227  94.92177707  98.6368467
 100.        ]


In [201]:
pca = PCA(n_components=11)
pca.fit(scaled_data)

PCA(n_components=11)

In [202]:
X = pca.transform(scaled_data)
y = y_train

In [203]:
from sklearn import linear_model
clf = linear_model.Ridge(alpha=0.0001,max_iter=800,tol = 0.000001)
clf.fit(X,y)

Ridge(alpha=0.0001, max_iter=800, tol=1e-06)

In [204]:
#x_test = x_test.drop(['position','name','age','club','nationality'], axis = 1)
scaled_data_test = scaler.fit_transform(x_test)
test_data = pca.transform(scaled_data_test)
pred = clf.predict(test_data)

In [205]:
import sklearn
sklearn.metrics.r2_score(y_test, pred, sample_weight=None, multioutput='uniform_average')

0.8326970193380239

In [206]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,pred)

24.998319302919665

In [207]:
pred_train = clf.predict(X)
mean_squared_error(y,pred_train)

36.89518793851001

In [208]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.0001,max_iter=800,tol = 0.000001)
clf.fit(X,y)
from sklearn.metrics import mean_squared_error
pred_lasso = clf.predict(test_data)
mean_squared_error(y_test,pred_lasso)

24.998789225119516

In [199]:
clf.n_iter_

2

In [200]:
import sklearn
sklearn.metrics.r2_score(y_test, pred_lasso, sample_weight=None, multioutput='uniform_average')

0.8174076279673361